<a href="https://colab.research.google.com/github/Amandaagambire/Galamsey-Watch/blob/main/G_watch_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Earth Engine**

In [ ]:
!pip install earthengine-api geemap folium rasterio
!pip install segmentation-models-pytorch

# **Authenticate & Initialize earth engine**

In [ ]:
from google.colab import drive
import ee
import geemap
import folium
import rasterio
from PIL import Image
import numpy as np
import os
import shutil
import pandas as pd
from skimage.color import rgb2gray
from skimage import exposure
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import label, regionprops
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms



# Mount Google Drive
try:
    drive.mount('/content/drive')
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

#Authenticae then initialise google Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-agamaamanda')
print("Earth Engine Initialized")


# **DATA ACQUISITION: Fetching Sentinel -2 Images**

In [ ]:

# set coordinates Region of interest  -> (Ghana)
region = ee.Geometry.Rectangle([-3.3, 4.5, 1.3, 11.2])  # (long, lat)


def preprocess_image(image):
      return image.select(['B4', 'B3', 'B2']).divide(10000) # Blue, Green, Red, NIR

def fetch_sentinel_image():
  # Get Sentinel-2 image collection
  sentinel = ee.ImageCollection("COPERNICUS/S2") \
              .filterBounds(region) \
              .filterDate('2024-01-01', '2024-12-31') \
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))\
              .map(preprocess_image) \
              .median() \
              # .clip(region) # Get just the region
  print("Sentinel-2 image preprocessed successfully!")
  return sentinel

  # Display metadata
  print(sentinel.getInfo())


# **Fetching Landsat-8 Images (UNNECESSARY?)**



In [ ]:

# Get Landsat-8 image collection
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(region) \
            .filterDate('2022-01-01', '2024-12-31') \
            .median()

# Select RGB bands
landsat_rgb = landsat.select(['SR_B4', 'SR_B3', 'SR_B2'])

# Print metadata
print(landsat.getInfo())


# **Vizulaize the Extracted images**

In [ ]:

Map = geemap.Map(center = [5.2, -2.1], zoom=10)

Map.addLayer(sentinel, {'min': 0, 'max': 3000}, "Sentinel-2 RGB")
Map.addLayer(landsat_rgb, {'min': 0, 'max': 3000}, "Landsat-8")

#Lets see it
Map

# **Export images to Google Drive**

In [ ]:
def export_image_to_drive(image, description='Sentinel-2_Mining_Area', folder = 'GEE_Exports'):
  export_task = ee.batch.Export.image.toDrive(
      image=image,
      description=description,
      scale=30,
      region=region,
      fileFormat='GeoTIFF',
      folder= folder  # This is where the file will be saved in Drive
  )
  # Start the export task
  export_task.start()
  print("Export started! Check Google Drive → GEE_Exports folder in 5-10 minutes.")



task_list = ee.batch.Task.list()
for task in task_list:
    print(task.status())




# **Moves a file from Google Drive to Colab**

In [ ]:
def move_file_from_drive_to_colab(source_path_on_drive, destination_path_on_colab):
    """Moves a file from Google Drive to Colab."""
    if IN_COLAB:
        source_path = f"/content/drive/My Drive/{source_path_on_drive}"
        try:
            shutil.copy(source_path, destination_path_on_colab)
            print(f"File moved to Colab: {destination_path_on_colab}")
            return True
        except FileNotFoundError:
            print(f"Error: Source file not found at {source_path}")
            return False
        except Exception as e:
            print(f"Error moving file: {e}")
            return False
    else:
        print("Not running in Colab, skipping file move.")
        return False

local_image_path = '/content/Sentinel2_Mining_Area.tif' # Assuming you have downloaded or moved the image


# **Download & Label for training**

In [ ]:
import os

# Check if file exists in Google Colab
print(os.listdir('/content/'))


In [ ]:
try:
  with rasterio.open(local_image_path) as src:
    image_array = src.read()
    image_metadata = src.profile
    red_band = image_array[0].astype(np.float32)
    green_band = image_array[1].astype(np.float32)
    blue_band = image_array[2].astype(np.float32)
    # nir_band = image_array[3]
except Exception as e:
    print(f"Error opening the image: {e}")

print(image_array.shape)
  # Print metadta
print(image_metadata)


# **U-Net Model Definition**

In [ ]:
from logging import log
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.inc = DoubleConv(in_channels, 64)
        self.outc = nn.Conv2d(64, out_channels, kernel_size=1) # Simplified output

    def forward(self, x):
        x1 = self.inc(x)
        logits = self.outc(x1)
        return logits

# Load a pre-trained model (replace with the actual path to your trained model)
n_channels = 3
n_classes = 1
model = UNet(n_channels, n_classes)
try:
    model.load_state_dict(torch.load('/content/drive/My Drive/galamsey_unet_simplified.pth', map_location=torch.device('cpu'))) # Load on CPU for simplicity
    model.eval()
    print("Pre-trained U-Net model loaded successfully!")
except FileNotFoundError:
    print("Error: Pre-trained U-Net model not found. Please train and save your model.")
    model = None

# Define image transformation for prediction
predict_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# **Prediction Function**

In [ ]:
def predict_mining_site(image_array, model, transform, device='cpu'):
    """
    Performs prediction on an image using the U-Net model.

    Args:
        image_array (numpy.ndarray): The input image array (e.g., RGB).
        model (torch.nn.Module): The trained U-Net model.
        transform (torch.transforms): The image transformation pipeline.
        device (str): 'cpu' or 'cuda'.

    Returns:
        numpy.ndarray or None: The binary prediction mask (1 for mining, 0 otherwise), or None if model is not loaded.
    """
    if model is None or image_array is None:
        print("Error: Model not loaded or image not available for prediction.")
        return None

    # Assuming image_array is (bands, height, width), we need (height, width, bands) for PIL
    rgb_image = np.transpose(image_array[:3, :, :], (1, 2, 0))
    pil_image = Image.fromarray(rgb_image).convert('RGB')
    input_tensor = transform(pil_image).unsqueeze(0).to(device) # Add batch dimension

    with torch.no_grad():
        output = model(input_tensor)
        prediction = torch.sigmoid(output).squeeze().cpu().numpy() > 0.5

    return prediction.astype(np.uint8)

# **Connected Components Analysis**

In [ ]:
def analyze_mining_clusters(prediction_mask):
    """
    Analyzes a binary prediction mask to detect clusters of illegal mining activity.

    Args:
        prediction_mask (numpy.ndarray): A binary mask where 1 represents mining.

    Returns:
        list: A list of dictionaries, where each dictionary contains information
              about a detected mining cluster (e.g., area, bounding box, centroid).
    """
    if prediction_mask is None:
        print("Error: No prediction mask available for cluster analysis.")
        return []

    labeled_mask = label(prediction_mask, connectivity=1)
    regions = regionprops(labeled_mask)
    mining_clusters = []
    for region in regions:
        if region.area > 10: # Example threshold
            mining_clusters.append({
                'area': region.area,
                'bounding_box': region.bbox,
                'centroid': region.centroid
            })
    return mining_clusters


# **Basic Risk Propagation (Conceptual)**

In [ ]:
def calculate_risk_propagation(elevation_data, mining_locations, water_sources):
    """
    A very basic conceptual implementation of risk propagation using Dijkstra's.
    This requires actual elevation data and water source locations.

    Args:
        elevation_data (numpy.ndarray): A 2D array representing elevation.
        mining_locations (list of tuples): Coordinates (row, col) of mining sites.
        water_sources (list of tuples): Coordinates (row, col) of water sources.

    Returns:
        dict: A dictionary mapping mining locations to the nearest water source path (conceptual).
    """
    if elevation_data is None or not mining_locations or not water_sources:
        print("Error: Missing data for risk propagation analysis.")
        return {}

    rows, cols = elevation_data.shape
    graph = nx.grid_2d_graph(rows, cols)

    # Define a very simple weight based on elevation (higher elevation = higher weight)
    for u, v, data in graph.edges(data=True):
        u_row, u_col = u
        v_row, v_col = v
        weight = (elevation_data[u_row, u_col] + elevation_data[v_row, v_col]) / 2 + 1
        data['weight'] = weight

    risk_paths = {}
    for mining_loc in mining_locations:
        shortest_path = None
        min_distance = float('inf')
        for water_loc in water_sources:
            try:
                path = nx.dijkstra_path(graph, source=mining_loc, target=water_loc, weight='weight')
                distance = nx.dijkstra_path_length(graph, source=mining_loc, target=water_loc, weight='weight')
                if distance < min_distance:
                    min_distance = distance
                    shortest_path = path
            except nx.NetworkXNoPath:
                pass

        if shortest_path:
            risk_paths[mining_loc] = f"Path to nearest water source for {mining_loc}" # Conceptual path

    return risk_paths

# Create dummy elevation data and water sources for demonstration
if image_array is not None:
    dummy_elevation_data = np.random.rand(image_array.shape[1], image_array.shape[2]) * 100
    dummy_water_sources = [(50, 50), (150, 200)] # Example coordinates